In [1]:
import os
import pandas as pd

#Le o csv de monossacarídeos
mon_df = pd.read_csv('/home/douglas/carboanalysis/carboanalysis/pdb/dataframes/SIC/monossaccharides.csv', names=['comp_id', 'entry_id', 'oligossaccharide', 'entity_id', 'comp_num', 'mol_num', 'name'])
 

In [2]:
print(len(mon_df['entry_id'].unique()))

15602


In [3]:
#cria df dizendo quantas entradas do pdb contém cada carboidrato
comp_ids = []
in_n_entries = []
names = []

for comp_id in mon_df['comp_id'].unique():
    comp_ids.append(comp_id)
    in_n_entries.append(len(mon_df['entry_id'].loc[mon_df['comp_id'] == comp_id].unique()))
    names.append(mon_df['name'].loc[mon_df['comp_id'] == comp_id].unique())


frequence_df = pd.DataFrame(data = {'comp_id': comp_ids, 'in_n_entries': in_n_entries, 'name': names})

In [4]:
#cria df dizendo quantas entradas do pdb contém cada carboidrato para as entradas filtradas
comp_ids_filtered = []
in_n_entries_filtered = []
names_filtered = []

#le a lista de entradas filtradas, pra comparação
filtered_entries= pd.read_csv("/home/douglas/carboanalysis/carboanalysis/pdb/dataframes/carbo_entrys_res_owab_filtered_nd.txt", names = ['entry_filename'])
filtered_entries['entry_filename'] = filtered_entries['entry_filename'].str.replace('.cif', '')
filtered_entries['entry_filename'] = filtered_entries['entry_filename'].apply(str.upper)

#Filtragem
mon_df_filtered = mon_df.loc[mon_df['entry_id'].isin(filtered_entries['entry_filename'])]

for comp_id_filtered in mon_df_filtered['comp_id'].unique():
    comp_ids_filtered.append(comp_id_filtered)
    in_n_entries_filtered.append(len(mon_df_filtered['entry_id'].loc[mon_df_filtered['comp_id'] == comp_id_filtered].unique()))
    names_filtered.append(mon_df_filtered['name'].loc[mon_df_filtered['comp_id'] == comp_id_filtered].unique())


frequence_df_filtered = pd.DataFrame(data = {'comp_id': comp_ids_filtered, 'in_n_entries': in_n_entries_filtered, 'name': names_filtered})

/tmp/ipykernel_990679/1131893877.py:8: FutureWarning: The default value of regex will change from True to False in a future version.
  filtered_entries['entry_filename'] = filtered_entries['entry_filename'].str.replace('.cif', '')


In [5]:
#Analise da distribuição da resolução nas entradas contendo carboidratos (n = 15619)

res_df = pd.read_csv('/home/douglas/carboanalysis/carboanalysis/pdb/dataframes/SIC/res.csv', names=['entry_id', 'res'], sep=';')

res_analysis_df = pd.DataFrame()

labels = ['R ≤ 1.0', '1.0 < R ≤ 1.5', '1.5 < R ≤ 2.0', '2.0 < R ≤ 2.5', '2.5 < R ≤ 3.5', '3.5 < R ≤ 5.0', 'R ≥ 5.0', 'Total']
values = [
    len(res_df['res'].loc[(res_df['res'] <= 1.0)]),
    len(res_df['res'].loc[(res_df['res'] > 1.0) & (res_df['res'] <= 1.5)]),
    len(res_df['res'].loc[(res_df['res'] > 1.5) & (res_df['res'] <= 2.0)]),
    len(res_df['res'].loc[(res_df['res'] > 2.0) & (res_df['res'] <= 2.5)]),
    len(res_df['res'].loc[(res_df['res'] > 2.5) & (res_df['res'] <= 3.5)]),
    len(res_df['res'].loc[(res_df['res'] > 3.5) & (res_df['res'] <= 5.0)]),
    len(res_df['res'].loc[(res_df['res'] > 5.0)]),
    15619
]
percents = [
    ((values[0]*100)/values[7]),
    ((values[1]*100)/values[7]),
    ((values[2]*100)/values[7]),
    ((values[3]*100)/values[7]),
    ((values[4]*100)/values[7]),
    ((values[5]*100)/values[7]),
    ((values[6]*100)/values[7]),
    100
]

res_distribution = pd.DataFrame(data = {'label': labels, 'value': values, 'percent': percents})


In [6]:
print(frequence_df_filtered['in_n_entries'].sum())

11507


In [7]:
# Análise tamanho

from collections import Counter

df = pd.read_csv('/home/douglas/carboanalysis/carboanalysis/pdb/dataframes/SIC/monossaccharides.csv', names=['comp_id', 'entry_id', 'is_oligossaccharide', 'entity_id', 'comp_num', 'mol_num', 'name'])

#Contagem monossac
tam_df = df.loc[(df['is_oligossaccharide'] == False)]

monossacharides = tam_df['mol_num'].sum()
print(monossacharides)

#Contagem dos oligo
oligo_df = df.loc[(df['is_oligossaccharide'] == True)]
#print(oligo_df)

sizes = []

entries = oligo_df['entry_id'].unique()

for entry in entries:
    entry_df = oligo_df.loc[oligo_df['entry_id'] == entry]

    entities = entry_df['entity_id'].unique()

    for entity in entities:
        entity_df = entry_df.loc[entry_df['entity_id'] == entity]
        
        for mol_num in range(entity_df['mol_num'].max()):
            sizes.append(entity_df['comp_num'].max())

print(sizes)

di = sizes.count(2.0)
print(Counter(sizes))
print(sum(sizes))
#{2.0: 12875, 3.0: 5924, 4.0: 2529, 5.0: 1806, 6.0: 1181, 7.0: 752, 8.0: 490, 9.0: 274, 10.0: 166, 11.0: 45, 12.0: 15, 17.0: 4, 16.0: 3, 18.0: 3, 26.0: 2, 14.0: 2, 15.0: 2, 13.0: 1}

38749
[5.0, 5.0, 2.0, 5.0, 2.0, 2.0, 2.0, 3.0, 4.0, 5.0, 5.0, 2.0, 7.0, 2.0, 2.0, 2.0, 2.0, 3.0, 3.0, 2.0, 2.0, 5.0, 5.0, 5.0, 6.0, 2.0, 2.0, 2.0, 5.0, 3.0, 3.0, 3.0, 3.0, 2.0, 10.0, 10.0, 7.0, 7.0, 7.0, 7.0, 4.0, 8.0, 3.0, 3.0, 3.0, 3.0, 3.0, 2.0, 2.0, 3.0, 3.0, 7.0, 9.0, 2.0, 2.0, 4.0, 2.0, 2.0, 5.0, 5.0, 6.0, 6.0, 5.0, 7.0, 2.0, 2.0, 2.0, 2.0, 2.0, 3.0, 3.0, 2.0, 2.0, 2.0, 2.0, 3.0, 3.0, 3.0, 3.0, 2.0, 2.0, 2.0, 2.0, 3.0, 2.0, 6.0, 8.0, 8.0, 4.0, 3.0, 3.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 10.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 4.0, 4.0, 3.0, 2.0, 2.0, 3.0, 5.0, 2.0, 6.0, 2.0, 2.0, 5.0, 5.0, 5.0, 4.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 7.0, 10.0, 3.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 3.0, 3.0, 3.0, 3.0, 2.0, 3.0, 6.0, 5.0, 2.0, 2.0, 5.0, 5.0, 3.0, 2.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 2.0, 4.0, 3.0, 3.0, 2.0, 4.0, 4.0, 2.0, 4.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0,

In [8]:
tam_dict = Counter(sizes)
oligo_size = pd.DataFrame(data = {'size': tam_dict.keys(), 'quantity': tam_dict.values()})

In [9]:
#Mesma coisa pra Res e OWAB filter

#le a lista de entradas filtradas, pra comparação
filtered_entries= pd.read_csv("/home/douglas/carboanalysis/carboanalysis/pdb/dataframes/carbo_entrys_res_owab_filtered_nd.txt", names = ['entry_filename'])
filtered_entries['entry_filename'] = filtered_entries['entry_filename'].str.replace('.cif', '')
filtered_entries['entry_filename'] = filtered_entries['entry_filename'].apply(str.upper)

df_filtered = df.loc[df['entry_id'].isin(filtered_entries['entry_filename'])]

#Contagem monossac
tam_df_filtered = df_filtered.loc[(df_filtered['is_oligossaccharide'] == False)]

monossacharides_filtered = tam_df_filtered['mol_num'].sum()
print(monossacharides_filtered)

#Contagem dos oligo
oligo_df_filtered = df_filtered.loc[(df_filtered['is_oligossaccharide'] == True)]
#print(oligo_df)

sizes_filtered = []

entries_filtered = oligo_df_filtered['entry_id'].unique()

for entry_filtered in entries_filtered:
    entry_df_filtered = oligo_df_filtered.loc[oligo_df_filtered['entry_id'] == entry_filtered]

    entities_filtered = entry_df_filtered['entity_id'].unique()

    for entity_filtered in entities_filtered:
        entity_df_filtered = entry_df_filtered.loc[entry_df_filtered['entity_id'] == entity_filtered]
        
        for mol_num_filtered in range(entity_df_filtered['mol_num'].max()):
            sizes_filtered.append(entity_df_filtered['comp_num'].max())

tam_dict_filtered = Counter(sizes_filtered)
oligo_size_filtered = pd.DataFrame(data = {'size': tam_dict_filtered.keys(), 'quantity': tam_dict_filtered.values()})

11816


/tmp/ipykernel_990679/373732839.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  filtered_entries['entry_filename'] = filtered_entries['entry_filename'].str.replace('.cif', '')


In [10]:
print(frequence_df['in_n_entries'].sum() - 8866 - 2816 - 2777 - 1857 - 1625 - 1440 - 1368 - 509)

6713
